In [1]:
from pymystem3 import Mystem
import sqlite3

In [2]:
#pip install pymystem3

In [3]:
m = Mystem()

In [4]:
conn = sqlite3.connect('poems_corpus.db')
cur = conn.cursor()

In [5]:
cur.execute("""
CREATE TABLE IF NOT EXISTS tokens (
    token_id INTEGER PRIMARY KEY AUTOINCREMENT,
    token TEXT,
    lemma TEXT,
    pos TEXT,
    sent_id int
)
""")

In [6]:
cur.execute('SELECT id_sent, sent FROM sentences')
for idx, sent in cur.fetchall():
    ana = m.analyze(sent)
    for word in ana:
        if 'analysis' in word:
            token = word['text'].lower()
            lemma = token
            pos = None
            if word['analysis']:
                lemma = word['analysis'][0]['lex']
                gr = word['analysis'][0]['gr']
                pos = gr.split('=')[0].split(',')[0]
                if 'прич' in gr:
                    pos = 'PRT'
                elif 'деепр' in gr:
                    pos = 'GRND'
            cur.execute(
                """
                INSERT INTO tokens 
                (token, lemma, pos, sent_id)
                VALUES (?, ?, ?, ?)
                """, (token, lemma, pos, idx)
            )

In [7]:
conn.commit()